## Import useful libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
import pytz

from airflow.providers.mysql.hooks.mysql import MySqlHook

## User settings

In [2]:
#list_of_price_types = ['bid', 'mid', 'ask']
#instrument_name = 'EUR/USD'
#interval_name = 'Day'

database_name = 'django'

## Connect to database

In [3]:
mysql_hook = MySqlHook(mysql_conn_id = database_name)

## Define a function to pull candlestick data

This only works for a single (price type, instrument, interval) tuple.

In [4]:
def get_candlestick_pull_query():
    sql_query_for_candlestick_pull = """SELECT

    ts.timestamp, inst.name AS instrument_name, cs.o, cs.l, cs.h, cs.c, v.volume

    FROM

    timeseries_candlestick cs, timeseries_instrument inst,
    timeseries_interval iv, timeseries_pricetype pt,
    timeseries_volume v, timeseries_timestamp ts

    WHERE

    cs.instrument_id = inst.id
    AND cs.interval_id = iv.id
    AND cs.price_type_id = pt.id
    AND cs.volume_id = v.id
    AND cs.timestamp_id = ts.id

    AND pt.name = '%s'
    AND inst.name = '%s'
    AND iv.name = '%s'

    ORDER BY timestamp
    ;
    """

    return sql_query_for_candlestick_pull

In [21]:
class CandlestickDataFrame():

    def __init__(
        self,
        instrument_name = 'EUR/USD',
        interval_name = 'Day',
        list_of_price_types = ['bid', 'mid', 'ask'],
        timezone_to_use = 'US/Eastern',
    ):
        
        self.list_of_price_types = list_of_price_types
        self.instrument_name = instrument_name
        self.interval_name = interval_name
        self.timezone_to_use = timezone_to_use

    def create_initial_dataframe(self):
        
        list_of_dataframes = []

        for pt in self.list_of_price_types:
            sql_to_run = get_candlestick_pull_query() % (pt, self.instrument_name, self.interval_name)
            df_pt = mysql_hook.get_pandas_df(sql_to_run)
    
            column_name_map = {}
            for col_name in ['o', 'l', 'h', 'c', 'volume']:
                column_name_map[col_name] = pt + '_' + col_name

            df_pt.rename(columns = column_name_map, inplace = True)

            list_of_dataframes.append(df_pt)

        df = list_of_dataframes[0]
        for df_i in list_of_dataframes[1:]:
            df = pd.merge(df, df_i, how = 'left', on = ['timestamp', 'instrument_name'])

        self.df = df

    def add_date_and_time_related_features(self):
        tz = pytz.timezone(self.timezone_to_use)
        self.df['timestamp_dt_Eastern'] = [datetime.datetime.fromtimestamp(x, tz = tz) for x in self.df['timestamp']]
        self.df['weekday_Eastern'] = [x.weekday() for x in self.df['timestamp_dt_Eastern']]
        self.df['hour_Eastern'] = [x.hour for x in self.df['timestamp_dt_Eastern']]

    def conduct_volume_test(self):
        volume_test = np.min(
            np.int8(
                (self.df['ask_volume'].values == self.df['mid_volume']) &
                (self.df['mid_volume'].values == self.df['bid_volume'])
            )
        )
        if volume_test == 1:
            self.df['volume'] = self.df['mid_volume']
            self.df.drop(columns = [x + '_volume' for x in self.list_of_price_types], inplace = True)

    def conduct_nan_test(self):
        self.are_there_nans = (len(self.df.index) != len(self.df.dropna().index))
    
    def fit(self):
        self.create_initial_dataframe()
        self.conduct_volume_test()
        self.add_date_and_time_related_features()
        self.conduct_nan_test()

In [22]:
cdf = CandlestickDataFrame()
cdf.fit()

In [23]:
cdf.df

,timestamp,instrument_name,bid_o,bid_l,bid_h,bid_c,mid_o,mid_l,mid_h,mid_c,ask_o,ask_l,ask_h,ask_c,volume,timestamp_dt_Eastern,weekday_Eastern,hour_Eastern
0,1020718800,EUR/USD,0.91520,0.91520,0.91520,0.91520,0.91535,0.91535,0.91535,0.91535,0.91550,0.91550,0.91550,0.91550,1,2002-05-06 17:00:00-04:00,0,17
1,1020805200,EUR/USD,0.90420,0.90420,0.90420,0.90420,0.90435,0.90435,0.90435,0.90435,0.90450,0.90450,0.90450,0.90450,1,2002-05-07 17:00:00-04:00,1,17
2,1020891600,EUR/USD,0.90920,0.90920,0.90920,0.90920,0.90935,0.90935,0.90935,0.90935,0.90950,0.90950,0.90950,0.90950,1,2002-05-08 17:00:00-04:00,2,17
3,1020978000,EUR/USD,0.91350,0.91350,0.91350,0.91350,0.91400,0.91400,0.91400,0.91400,0.91450,0.91450,0.91450,0.91450,1,2002-05-09 17:00:00-04:00,3,17
4,1021064400,EUR/USD,0.91360,0.91360,0.91360,0.91360,0.91410,0.91410,0.91410,0.91410,0.91460,0.91460,0.91460,0.91460,1,2002-05-10 17:00:00-04:00,4,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6764,1742763600,EUR/USD,1.08138,1.07808,1.08576,1.08004,1.08156,1.07816,1.08583,1.08012,1.08174,1.07823,1.08591,1.08020,136246,2025-03-23 17:00:00-04:00,6,17
6765,1742850000,EUR/USD,1.08004,1.07763,1.08295,1.07905,1.08013,1.07770,1.08302,1.07918,1.08022,1.07777,1.08310,1.07930,115335,2025-03-24 17:00:00-04:00,0,17
6766,1742936400,EUR/USD,1.07890,1.07432,1.08021,1.07518,1.07940,1.07439,1.08029,1.07528,1.07990,1.07446,1.08037,1.07539,124703,2025-03-25 17:00:00-04:00,1,17
6767,1743022800,EUR/USD,1.07552,1.07322,1.08205,1.08011,1.07578,1.07330,1.08212,1.08020,1.07605,1.07338,1.08221,1.08030,131881,2025-03-26 17:00:00-04:00,2,17


In [24]:
cdf.are_there_nans

False

In [18]:
(
    cdf
    .df
    .groupby('hour_Eastern')
    ['timestamp']
    .agg('count')
)

hour_Eastern
17    6769
Name: timestamp, dtype: int64